In [188]:
import networkx as nx
import numpy as np

In [189]:
G=nx.Graph()
G.add_nodes_from([0,1,2,3,4])
G.add_edges_from([(1,4), (0,4), (2,3)])

In [190]:
from cc_model.wl import WL

In [191]:
_, wl_colors = WL(G)

In [192]:
A=np.array(nx.adjacency_matrix(G).todense())
A

array([[0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0],
       [1, 1, 0, 0, 0]])

In [193]:
def H(v, normalize=False):
    n_rows = len(v)
    n_cols = v.max()+1
    
    H = np.zeros((n_rows, n_cols), dtype=int)
    print(H.shape)
    for i,j in enumerate(v):
        H[i,j]=1
    if not normalize:
        return H
    else:
        H_out = np.zeros(H.shape)
        for i in range(n_cols):
            x = np.sqrt(H[:,i].sum())
            print(x)
            H_out[:,i]=H[:,i]/x
        return H_out

In [194]:
H0 = H(wl_colors[0], normalize=True)
H1 = H(wl_colors[1], normalize=True)
H2 = H(wl_colors[2], normalize=True)
print(H1)

H1_ = H(wl_colors[1], normalize=False)
H2_ = H(wl_colors[2], normalize=False)

(5, 1)
2.23606797749979
(5, 2)
2.0
1.0
(5, 3)
1.4142135623730951
1.4142135623730951
1.0
[[0.5 0. ]
 [0.5 0. ]
 [0.5 0. ]
 [0.5 0. ]
 [0.  1. ]]
(5, 2)
(5, 3)


In [195]:
H1.T@H1

array([[1., 0.],
       [0., 1.]])

In [196]:
H2

array([[0.70710678, 0.        , 0.        ],
       [0.70710678, 0.        , 0.        ],
       [0.        , 0.70710678, 0.        ],
       [0.        , 0.70710678, 0.        ],
       [0.        , 0.        , 1.        ]])

In [197]:
def to_vector(d):
    v = np.empty(len(d))
    for key, value in d.items():
        v[key]=value
    return v

In [198]:
v = to_vector(nx.eigenvector_centrality(G))

In [199]:
v

array([5.00000000e-01, 5.00000000e-01, 1.06289242e-05, 1.06289242e-05,
       7.07106781e-01])

In [200]:
v-H1@H1.T@v

array([ 0.24999469,  0.24999469, -0.24999469, -0.24999469,  0.        ])

In [201]:
v-H2@H2.T@v

array([1.11022302e-16, 1.11022302e-16, 1.69406589e-21, 1.69406589e-21,
       0.00000000e+00])

In [202]:
A.T@H1_

array([[0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [2, 0]])

In [203]:
A2=np.zeros((3,2), dtype=int)
A2[0,1]=1

A2[1,0]=1
A2[2,0]=2
print(A2)
H2_@A2

[[0 1]
 [1 0]
 [2 0]]


array([[0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [2, 0]])

In [204]:
H2.T@np.ones((5,1))

array([[1.41421356],
       [1.41421356],
       [1.        ]])

In [205]:
H2_.shape

(5, 3)

In [206]:
A@np.ones((5,1))

array([[1.],
       [1.],
       [1.],
       [1.],
       [2.]])

In [207]:
H0@H0.T

array([[0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2],
       [0.2, 0.2, 0.2, 0.2, 0.2]])

In [208]:
H1@H1.T

array([[0.25, 0.25, 0.25, 0.25, 0.  ],
       [0.25, 0.25, 0.25, 0.25, 0.  ],
       [0.25, 0.25, 0.25, 0.25, 0.  ],
       [0.25, 0.25, 0.25, 0.25, 0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  ]])

In [209]:
H1.T@H1

array([[1., 0.],
       [0., 1.]])

In [210]:
import numpy as np

def arnoldi_iteration(A, b, n: int):
    """Computes a basis of the (n + 1)-Krylov subspace of A: the space
    spanned by {b, Ab, ..., A^n b}.

    Arguments
      A: m × m array
      b: initial vector (length m)
      n: dimension of Krylov subspace, must be >= 1
    
    Returns
      Q: m x (n + 1) array, the columns are an orthonormal basis of the
        Krylov subspace.
      h: (n + 1) x n array, A on basis Q. It is upper Hessenberg.  
    """
    eps = 1e-12
    h = np.zeros((n+1,n))
    Q = np.zeros((A.shape[0],n+1))
     # Normalize the input vector
    Q[:,0] =b/np.linalg.norm(b,2)   # Use it as the first Krylov vector
    for k in range(1,n+1):
        v = np.dot(A,Q[:,k-1])  # Generate a new candidate vector
        for j in range(k):  # Subtract the projections on previous vectors
            h[j,k-1] = np.dot(Q[:,j].T, v)
            v = v - h[j,k-1] * Q[:,j]
        h[k,k-1] = np.linalg.norm(v,2)
        if h[k,k-1] > eps:  # Add the produced vector to the list, unless
            Q[:,k] = v/h[k,k-1]
        else:  # If that happens, stop iterating.
            return Q, h
    return Q, h

In [211]:
arnoldi_iteration(A, np.ones(A.shape[0]), 3)

(array([[ 4.47213595e-01, -2.23606798e-01,  5.00000000e-01,
          0.00000000e+00],
        [ 4.47213595e-01, -2.23606798e-01,  5.00000000e-01,
          0.00000000e+00],
        [ 4.47213595e-01, -2.23606798e-01, -5.00000000e-01,
          0.00000000e+00],
        [ 4.47213595e-01, -2.23606798e-01, -5.00000000e-01,
          0.00000000e+00],
        [ 4.47213595e-01,  8.94427191e-01,  1.98602732e-16,
          0.00000000e+00]]),
 array([[ 1.20000000e+00,  4.00000000e-01,  3.88578059e-16],
        [ 4.00000000e-01, -7.00000000e-01,  1.11803399e+00],
        [ 0.00000000e+00,  1.11803399e+00,  5.00000000e-01],
        [ 0.00000000e+00,  0.00000000e+00,  5.48159155e-16]]))

In [212]:
np.linalg.eigh(A)

(array([-1.41421356e+00, -1.00000000e+00, -6.07153217e-18,  1.00000000e+00,
         1.41421356e+00]),
 array([[-0.5       , -0.        , -0.70710678,  0.        ,  0.5       ],
        [-0.5       , -0.        ,  0.70710678,  0.        ,  0.5       ],
        [ 0.        ,  0.70710678,  0.        , -0.70710678,  0.        ],
        [ 0.        , -0.70710678,  0.        , -0.70710678,  0.        ],
        [ 0.70710678,  0.        ,  0.        ,  0.        ,  0.70710678]]))

In [217]:
V,B = np.linalg.eigh(H2.T@A@H2)
print(V)

[-1.41421356  1.          1.41421356]


In [218]:
H2@B

array([[-0.5       ,  0.        , -0.5       ],
       [-0.5       ,  0.        , -0.5       ],
       [ 0.        , -0.70710678,  0.        ],
       [ 0.        , -0.70710678,  0.        ],
       [ 0.70710678,  0.        , -0.70710678]])

In [216]:
H1.T@H1

array([[1., 0.],
       [0., 1.]])